One way to test for a warming climate is to measure the number of days above and below average over a time period. If temperature were not increasing, a reasonable conclusion is that the number of days above and below average would be roughly equal over long enough time periods. By contrast, a warming climate would see more days above average than below. We can calculate this by checking each day against a 30 year rolling average for that particular day in year and measuring the deviation from this number.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
whitestown = pd.read_csv('C:\csv\whitestownweather.csv')

cols = ['DATE', 'TMAX', 'TMIN']

whitestown = whitestown[cols]
whitestown['DATE'] = pd.to_datetime(whitestown['DATE'])
whitestown['YEAR'] = whitestown['DATE'].dt.year
whitestown['DOY'] = whitestown['DATE'].dt.dayofyear # day of year
whitestown.head()

,DATE,TMAX,TMIN,YEAR,DOY
0,1901-01-01,20.0,-4.0,1901,1
1,1901-01-02,21.0,-12.0,1901,2
2,1901-01-03,28.0,-6.0,1901,3
3,1901-01-04,40.0,3.0,1901,4
4,1901-01-05,35.0,18.0,1901,5


Let's start with 1969. We will slice this data out but still use the original dateframe to calculate with (because we need 1938-1968).

In [35]:
whitestown_slice = whitestown[whitestown['YEAR'] > 1990]
whitestown_slice.head()

,DATE,TMAX,TMIN,YEAR,DOY
32837,1991-01-01,34.0,14.0,1991,1
32838,1991-01-02,34.0,19.0,1991,2
32839,1991-01-03,28.0,16.0,1991,3
32840,1991-01-04,29.0,9.0,1991,4
32841,1991-01-05,34.0,23.0,1991,5


In [44]:
anom_list = []

for index, row in whitestown_slice.iterrows():
    start = row[3] - 30
    data_slice = whitestown[whitestown['DOY'] == row[4]]
    data_slice = data_slice[data_slice['YEAR'] > start]
    data_slice = data_slice[data_slice['YEAR'] < row[3]]
    
    average = data_slice['TMAX'].mean()
    anomaly = row[1] - average
    
    anom_list.append(anomaly)
    
whitestown_slice['ANOMALY'] = anom_list

C:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
whitestown_pos = whitestown_slice[whitestown_slice['ANOMALY'] > 0]
whitestown_neg = whitestown_slice[whitestown_slice['ANOMALY'] < 0]

In [46]:
whitestown_pos.shape[0]

5249

In [47]:
whitestown_neg.shape[0]

4947

So 5249 days were above average and 4947 were below average from 1991-2018. To test the significance of this result we can use a chi-square test.

In [50]:
from scipy.stats import chisquare

expected = [5098, 5098]
observed = [5249, 4947]

chisquare(observed, expected)

Power_divergenceResult(statistic=8.945076500588467, pvalue=0.0027821844271509574)